## Import the required libraries and set any costants being used

In [1]:
# Standard libraries
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)

# PyTorch
import torch

# HuggingFace datasets
from datasets import Dataset, DatasetDict, Audio, load_from_disk, concatenate_datasets

# HuggingFace transformers
from transformers import (
    WhisperFeatureExtractor,
    WhisperTokenizer,
    WhisperProcessor,
    WhisperForConditionalGeneration,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer
)

# Python standard library and other external imports
from dataclasses import dataclass
from typing import Any, Dict, List, Union

# Local modules
import evaluate

BASE_MODEL="openai/whisper-tiny"
LANGUAGE="bengali"

# Load the data from huggingface.co


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [2]:
from datasets import load_dataset
ds = load_dataset("thesven/bengali-ai-train-set-tiny")
ds['train'][0]

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating valid split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /Users/michaelsvendsen/.cache/huggingface/datasets/thesven___parquet/thesven--bengali-ai-train-set-tiny-f69ea9cd704be2a7/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

{'input_features': [[-0.6540145874023438,
   -0.6540145874023438,
   -0.6540145874023438,
   -0.6540145874023438,
   -0.2295091152191162,
   -0.07681560516357422,
   -0.2483837604522705,
   -0.6540145874023438,
   -0.6540145874023438,
   -0.6540145874023438,
   -0.6495187282562256,
   -0.6540145874023438,
   -0.6540145874023438,
   -0.6540145874023438,
   -0.6540145874023438,
   -0.6540145874023438,
   -0.6540145874023438,
   -0.2254241704940796,
   0.4447900056838989,
   0.41018134355545044,
   0.859327495098114,
   0.5746658444404602,
   0.6863173842430115,
   0.623473584651947,
   0.5883538126945496,
   0.17869430780410767,
   0.3296905755996704,
   0.3185269236564636,
   0.9516028761863708,
   0.8877100944519043,
   0.4896426200866699,
   0.5367220640182495,
   0.7518444657325745,
   0.6922017335891724,
   -0.08898341655731201,
   0.2541205883026123,
   0.42352086305618286,
   0.38110536336898804,
   0.1925731897354126,
   0.18576234579086304,
   0.0976981520652771,
   -0.001330614

In [8]:
feature_extractor = WhisperFeatureExtractor.from_pretrained(BASE_MODEL)
tokenizer = WhisperTokenizer.from_pretrained(BASE_MODEL, language=LANGUAGE, task="transcribe")

In [4]:
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch


## Create a whisper processor

In [5]:
processor = WhisperProcessor.from_pretrained(BASE_MODEL, language=LANGUAGE, task="transcribe")
print(processor)
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)
print(data_collator)

WhisperProcessor:
- feature_extractor: WhisperFeatureExtractor {
  "chunk_length": 30,
  "feature_extractor_type": "WhisperFeatureExtractor",
  "feature_size": 80,
  "hop_length": 160,
  "n_fft": 400,
  "n_samples": 480000,
  "nb_max_frames": 3000,
  "padding_side": "right",
  "padding_value": 0.0,
  "processor_class": "WhisperProcessor",
  "return_attention_mask": false,
  "sampling_rate": 16000
}

- tokenizer: WhisperTokenizer(name_or_path='openai/whisper-tiny', vocab_size=50258, model_max_length=1024, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|endoftext|>', '<|startoftranscr

## Load Pre-trained checkpoint

In [ ]:
model = WhisperForConditionalGeneration.from_pretrained(BASE_MODEL)
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

## Define Training Arguments

In [ ]:
metric = evaluate.load("wer")

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-tiny-bn-thesven",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=4000,
    gradient_checkpointing=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=ds['train'],
    eval_dataset=ds['valid'],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

## Train

In [ ]:
print(trainer.accelerator.device)

In [ ]:
trainer.train()

In [ ]:
processor.push_to_hub("whisper-medium-bn")
tokenizer.push_to_hub("whisper-medium-bn")